# 1. Load data
Load cvs file and retrieve bar and restaurants' names and addresses

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd

pd.options.display.max_rows = 30

In [ ]:
bars=pd.read_csv('Mixed_Beverage_Gross_Receipts.csv')  

In [ ]:
bars.columns

In [ ]:
bars2=bars[['Location Name','Location Address','Location City','Location State','Location Zip','Location County']]

In [ ]:
bars2.tail(10)
# bars2['Location Name'].astype(str) + bars2['Location Address']

In [ ]:
bars3 = bars2.groupby(['Location Name', 'Location Address','Location State','Location City','Location Zip']).size().reset_index(name='Freq')

In [ ]:
bars3.tail(10000)

In [ ]:
bars3['Google'] = bars3.loc[:,'Location Name'].astype(str).str.cat(bars3.loc[:,'Location Zip'].astype(str), sep=' \"')
bars3['Google'] = bars3.loc[:,'Google'].astype(str).str.cat(bars3.loc[:,'Location Address'].astype(str), sep='\" ')

In [ ]:
bars3.tail(10000)

# 2. Search in Google

In [ ]:
recs = []
errs = []
search=bars3.iloc[0,5]:
#search=bar+' tripadvisor'
req = requests.get('https://www.google.com/search',
              params = {'q': search},
              headers = {'User-Agent': ua},
              )
sleep(0.05)
try:
    yelpdata=re.search(r'Search Results.+?Web results.+?href=.+?www.yelp.com.+?</span></div><div.+?><div.+?>', req.text).group(0)
    price=re.search(r'Price range: (.+?)</span>', yelpdata).group(1)  
    rating=re.search(r'Rating: (.+?)</span>', yelpdata).group(1)
    web=re.search(r'href="(.+?)"', yelpdata).group(1)
    name=re.search(r'<span>(.+?)</span></h3><div class=', yelpdata).group(1)
    reviews=re.search(r'Rating.+?<span>(.+?) reviews*</span>', yelpdata).group(1)
    yelp_web=re.search(r'<a href="https://www.yelp.com(.+?)" data-ved=.+?ping=.+?', yelpdata).group(1)
    rec= bar, price, rating, reviews, web, yelp_web
    recs.append(rec)
    print(rec)
except:
    errs.append(bar)
    print('No data for', bar)

# 3. Search in Yelp

In [ ]:
bars3 = bars2.groupby(['Location Name', 'Location Address','Location State','Location City','Location Zip']).size().reset_index(name='Freq')
bars3['Location Name']=bars3['Location Name'].str.replace('\#','', regex=True)
bars3['Location Name']=bars3['Location Name'].str.replace('&','+', regex=True)
bars3['Location Name']=bars3['Location Name'].str.replace('\'','', regex=True)
bars3['Location Name']=bars3['Location Name'].str.replace(' ','+', regex=True)
bars3['Location City']=bars3['Location City'].str.replace(' ','+', regex=True)
#bars3['Location City']='Austin'

In [ ]:
bars3['Location City'].value_counts()

In [ ]:
bars3['Location City'].describe()

In [ ]:
search='https://www.yelp.com/search?find_desc=red+mesa+grill&find_loc=big+spring&ns=1'
print(search)
req=requests.get(search,
                    headers = {'User-Agent': ua,
                    'accept': '*/*',
                    'accept-encoding': 'gzip, deflate, br',
                    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
                    'content-type': 'application/json',
                    'dnt': '1',
                    'sec-fetch-dest': 'empty',
                    'sec-fetch-mode': 'cors',
                    'sec-fetch-site': 'same-origin'}
                    )
sleep(1)



In [ ]:
req.text

In [ ]:
yelpdata=re.search(r'>Top match<(.+?)>More businesses in similar categories<', req.text,re.DOTALL).group(1)
style=re.findall(r'find_loc.+?target=.+?>(?!<img class)(.+?)</a>', yelpdata)
style
rating=re.search(r'aria-label="(\d\.*\d*) star rating" role="img"', yelpdata).group(1)
rating
reviews=re.search(r'reviewCount.+?>(\d+)</span></div></div></div>', yelpdata).group(1)
reviews
web=re.search(r'href="(.+?)" target', yelpdata).group(1)
web
name=re.search(r'businessName.+?target=.+?name=.+?rel=.+?>(?!<img class)(.+?)</a></span></h4></div></div></div>', yelpdata,re.DOTALL).group(1)
name
#rec= bar, name, city, style, rating, reviews, web

In [ ]:
recs = []
errs = []
count = 0
bar=bars3.iloc[0,0]
city=bars3.iloc[0,3]
if city=='AUSTIN':
    count=count+1
    search='https://www.yelp.com/search?find_desc='+bar+'&find_loc='+city+'&ns=1'
    print(search)
    req=requests.get(search,
                headers = {'User-Agent': ua,
                'accept': '*/*',
                'accept-encoding': 'gzip, deflate, br',
                'accept-language': 'en-US,en;q=0.9,es;q=0.8',
                'content-type': 'application/json',
                'dnt': '1',
                'sec-fetch-dest': 'empty',
                'sec-fetch-mode': 'cors',
                'sec-fetch-site': 'same-origin'}
                )
    sleep(1)
    try:
        yelpdata=re.search(r'>Top match<(.+?)>More businesses in similar categories<', req.text,re.DOTALL).group(1)
        #price=re.search(r'priceRange.+?>(.+?)</span></span><span class', yelpdata).group(1)
        style=re.findall(r'find_loc.+?target=.+?>(?!<img class)(.+?)</a>', yelpdata)
        rating=re.search(r'aria-label="(\d\.*\d*) star rating" role="img"', yelpdata).group(1)
        reviews=re.search(r'reviewCount.+?>(\d+)</span></div></div></div>', yelpdata).group(1)
        web=re.search(r'href="(.+?)" target', yelpdata).group(1)
        name=re.search(r'businessName.+?target=.+?name=.+?rel=.+?>(?!<img class)(.+?)</a></span></h4></div></div></div>', yelpdata,re.DOTALL).group(1)
        rec= bar, name, city, style, rating, reviews, web
        recs.append(rec)
        print(rec)
    except:
            try:
                yelpdata=re.search(r'>All Results<.+?srcSet.+?srcSet', req.text).group(0)
                #price=re.search(r'priceRange.+?>(.+?)</span></span><span class', yelpdata).group(1)
                style=re.findall(r'find_loc.+?target=.+?>(?!<img class)(.+?)</a>', yelpdata)
                rating=re.search(r'aria-label="(\d\.*\d*) star rating" role="img"', yelpdata).group(1)
                reviews=re.search(r'reviewCount.+?>(\d+)</span></div></div></div>', yelpdata).group(1)
                web=re.search(r'href="(.+?)" target', yelpdata).group(1)
                name=re.search(r'businessName.+?target=.+?name=.+?rel=.+?>(?!<img class)(.+?);.+?</a></span></h4></div></div></div><d', yelpdata).group(1)
                rec= bar, name, city, style, rating, reviews, web
                recs.append(rec)
                print(rec)

            except:
                errs.append(bar)
                print('No data for', bar)


    #else:  
         #print('Not Austin')